> ⚠️ **작업 중 (Work in Progress)**: 이 노트북은 현재 개발 중입니다. 일부 코드가 불완전하거나 변경될 수 있습니다.

## 📋 목차

- [모델 탐색 (Discover)](#모델-탐색-discover)
- [모델 비교 및 배포](#모델-비교-및-배포)
- [Embedding 모델 배포](#embedding-모델-배포)
- [Model Router 배포](#model-router-배포)
- [Model Router 구성](#model-router-구성)

## 🎯 학습 목표

- 모델 리더보드를 통한 모델 성능 비교
- 다양한 AI 모델 배포 방법 이해
- Model Router 설정 및 구성
- 모델 라우팅 전략 이해

## ⏱️ 예상 소요 시간

약 15분

## 환경 설정

먼저 이전 노트북에서 생성한 Foundry 리소스 정보를 설정합니다.

In [ ]:
# 환경 변수 로드
import json
import os
import subprocess

# PATH 환경변수 설정 (Azure CLI를 찾을 수 있도록)
possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / GitHub Codespaces
    "/home/linuxbrew/.linuxbrew/bin"  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
except:
    pass

paths_to_add = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    new_path = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    os.environ["PATH"] = new_path

# 이전 노트북에서 저장한 설정 파일 로드
config_file = ".foundry_config.json"
try:
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # 환경 변수 설정
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    LOCATION = config.get("LOCATION")
    TENANT_ID = config.get("TENANT_ID")
    PROJECT_NAME = config.get("PROJECT_NAME", "proj-default")
    PROJECT_ENDPOINT = config.get("FOUNDRY_ENDPOINT")
    
    # 환경 변수로도 설정 (다른 도구들이 사용할 수 있도록)
    os.environ["FOUNDRY_NAME"] = FOUNDRY_NAME
    os.environ["LOCATION"] = LOCATION
    os.environ["RESOURCE_GROUP"] = RESOURCE_GROUP
    os.environ["AZURE_SUBSCRIPTION_ID"] = config.get("AZURE_SUBSCRIPTION_ID", "")
    os.environ["_ENDPOINT"] = PROJECT_ENDPOINT
    
    print(f"✅ 설정 파일 '{config_file}'에서 환경 변수를 로드했습니다.")
    print(f"\n📌 Foundry Name: {FOUNDRY_NAME}")
    print(f"📌 Resource Group: {RESOURCE_GROUP}")
    print(f"📌 Location: {LOCATION}")
    print(f"📌 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")
    
except FileNotFoundError:
    print(f"⚠️ '{config_file}' 파일을 찾을 수 없습니다.")
    print("💡 01-setup.ipynb를 먼저 실행하여 환경을 설정하세요.")
    raise

# 필수 패키지 설치
%pip install -q azure-ai-projects azure-identity

from azure.ai.projects import AIProjectClient
from azure.identity import InteractiveBrowserCredential

print(f"\n💡 사용할 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")

## 사용 가능한 모델 조회

Azure CLI로 Sweden Central 리전에서 사용 가능한 모델을 확인합니다.

In [ ]:
# Sweden Central 리전에서 사용 가능한 모델 조회
print("🔍 Sweden Central 리전에서 사용 가능한 AI 모델 조회 중...")
print("=" * 80)

# Azure CLI로 리전에서 사용 가능한 모델 조회
# GPT 관련 모델 필터링하여 보기 쉽게 표시
import subprocess

cmd = [
    "az", "cognitiveservices", "model", "list",
    "--location", LOCATION,
    "--query", "[?contains(model.name, 'gpt') || contains(model.name, 'embedding')].{Name:model.name, Version:model.version, Format:model.format, Kind:kind}",
    "--output", "table"
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)

if result.stderr and not result.stderr.startswith("("):  # Azure CLI warnings 무시
    print(result.stderr)

print("\n" + "=" * 80)
print("💡 전체 모델 목록을 보려면 아래 명령어를 실행하세요:")
print(f"   !az cognitiveservices model list --location {LOCATION} --output table")
print("\n💡 위 목록에서 원하는 모델을 선택하여 아래 셀에서 배포할 수 있습니다.")
print("💡 모델 이름(Name)과 버전(Version)을 확인하세요.")

## GPT-4.1 모델 배포

GPT-4.1 모델을 배포합니다. GlobalStandard SKU를 사용하여 최적의 성능을 제공합니다.

In [ ]:
# GPT-4.1 모델 배포
!az cognitiveservices account deployment create \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --deployment-name gpt-4.1 \
    --model-name gpt-4.1 \
    --model-format OpenAI \
    --model-version "2025-04-14" \
    --sku-name GlobalStandard \
    --sku-capacity 1

print("\n✅ GPT-4.1 모델 배포 완료!")

## GPT-5.1 모델 배포

GPT-5.1 모델을 배포합니다. GlobalStandard SKU를 사용하여 최적의 성능을 제공합니다.

In [ ]:
# GPT-5.1 모델 배포
!az cognitiveservices account deployment create \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --deployment-name gpt-5.1 \
    --model-name gpt-5.1 \
    --model-format OpenAI \
    --model-version "2025-11-13" \
    --sku-name GlobalStandard \
    --sku-capacity 1

print("\n✅ GPT-5.1 모델 배포 완료!")

## 배포 상태 확인

배포된 모델의 상태를 확인합니다.

In [ ]:
# 배포 상태 확인
!az cognitiveservices account deployment show \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --deployment-name gpt-4.1 \
    --query "{Name:name, Model:properties.model.name, Version:properties.model.version, Status:properties.provisioningState}" \
    --output table

# 모든 배포된 모델 목록
print("\n📦 배포된 모든 모델:")
!az cognitiveservices account deployment list \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --output table

## Embedding 모델 배포

Embedding 모델은 텍스트를 벡터로 변환하여 의미적 검색 및 유사도 계산에 사용됩니다.

In [ ]:
# text-embedding-3-large 모델 배포 (벡터 차원: 3072)
!az cognitiveservices account deployment create \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --deployment-name text-embedding-3-large \
    --model-name text-embedding-3-large \
    --model-format OpenAI \
    --model-version "1" \
    --sku-name GlobalStandard \
    --sku-capacity 1

print("\n✅ Embedding 모델 배포 완료!")

# 배포 상태 확인
!az cognitiveservices account deployment show \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --deployment-name text-embedding-3-large \
    --query "{Name:name, Model:properties.model.name, Status:properties.provisioningState}" \
    --output table
print("   벡터 차원: 3072 (Knowledge Base에서 사용)")

## Model Router 배포

Model Router는 여러 모델 간의 지능형 라우팅을 제공하여 비용, 품질, 성능을 최적화합니다.

### Routing Mode 옵션:

**a) Balanced Mode (균형 모드)** - 기본값
- 비용, 품질, 성능의 균형 유지
- 일반적인 프로덕션 워크로드에 적합

**b) Quality Mode (품질 모드)**
- 최고 품질의 응답 우선
- 정확도가 중요한 애플리케이션

**c) Cost Mode (비용 모드)**
- 비용 최적화 우선
- 대량의 간단한 요청 처리

In [ ]:
# Model Router 배포
!az cognitiveservices account deployment create \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --deployment-name model-router \
    --model-name model-router \
    --model-format OpenAI \
    --model-version "2025-11-18" \
    --sku-name GlobalStandard \
    --sku-capacity 1

print("\n✅ Model Router 배포 완료!")
print("\n📊 Model Router 동작 방식:")
print("  사용자 요청 → Model Router → 판단:")
print("    - 간단한 질문 → 저비용 모델")
print("    - 복잡한 분석 → 고품질 모델")
print("    - 높은 부하 → 부하 분산")

## 배포된 모델 최종 확인

모든 모델이 정상적으로 배포되었는지 확인합니다.

In [ ]:
# 최종 배포 목록 확인
print("=" * 80)
print("배포된 모델 목록")
print("=" * 80)

!az cognitiveservices account deployment list \
    --resource-group $RESOURCE_GROUP \
    --name $FOUNDRY_NAME \
    --query "[].{Name:name, Model:properties.model.name, SKU:sku.name, Status:properties.provisioningState}" \
    --output table

print("\n✅ 다음 모델이 배포되어야 합니다:")
print("  1. gpt-4.1 (Language Model)")
print("  2. gpt-5.1 (Language Model)")
print("  3. text-embedding-3-large (Embedding Model)")
print("  4. model-router (Router)")

print("\n💡 포털 확인: https://ai.azure.com")
print("   Build > Models에서 배포된 모델을 시각적으로 확인할 수 있습니다.")

## 다음 단계

모델 배포가 완료되었습니다! 이제 이 모델들을 활용하여 에이전트를 구축해봅시다:

➡️ **[03. 에이전트 개발](./03-agents.ipynb)**: 다양한 기능을 가진 AI 에이전트를 만들어봅니다.